In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
'''
numpy is going to complain about not being able to determine a dtype
when aggregating, but it can be safely ignored
'''
np.warnings.filterwarnings('ignore')

In [14]:
loc = './Data_Mashups/completed/building_complaints_equipment_and_compliance/NYCHPD_feeCharges/Fee_Charges.csv'
transactions_df = pd.read_csv(loc)

In [4]:

#add leading zeros to the bloack and lot so they concat with boro into standard 10 digit
transactions_df['Block'] = transactions_df['Block'].apply(lambda x: '{0:0>5}'.format(x))
transactions_df['Lot'] = transactions_df['Lot'].apply(lambda x: '{0:0>4}'.format(x))

#make BBL column
transactions_df['BBL'] = transactions_df['BoroID'].map(str) + transactions_df['Block'].map(str) + transactions_df['Lot'].map(str)



In [5]:
'''
put the date in the correct format. Explicitly declaring the 
format will help catch data integrity problems.
'''

transactions_df['DoFTransferDate'] = pd.to_datetime(
    transactions_df['DoFTransferDate'], format="%d/%m/%Y")

'''
For this column, there were some problems. 
Manual inspection revealed that dates are in mixed format.
Removed the explicit format and let pandas try to find the best match.
'''
transactions_df['FeeIssuedDate'] = pd.to_datetime(transactions_df['FeeIssuedDate'])

In [6]:
#get total fees per bbl
f1 = transactions_df.groupby('BBL').agg({'FeeAmount': np.sum})

#convert to int
f1['FeeAmount'] = f1['FeeAmount'].astype('int64')

#name reflects data transformation
f1.columns = ['TotalFeeAmount']

#get number of fees per building
f2 = transactions_df.groupby('BBL').size()

#combine the two and start building the intermediate data object
f1['NumberOfComplaints'] = f2


In [7]:
'''
Get the most common fee type for each building. Listed as a tuple of lists:
format ([name],[count])
'''

f3 = transactions_df.groupby('BBL').agg({'FeeSourceType': stats.mode})

#split the tuple into data friendlier for CSV

f1['ModeFeeTypeName'] = f3
f1['ModeFeeTypeCount'] = [i.count[0] for i in f1['ModeFeeTypeName']]
f1['ModeFeeTypeName'] = [i.mode[0] for i in f1['ModeFeeTypeName']]

In [8]:
#Get the lifecycle of the building. Similarly, format ([name],[count])

f4 = transactions_df.groupby('BBL').agg({'LifeCycle': stats.mode})

#split the tuple into data friendlier for CSV

f1['ModeLifeCycleName'] = f4
f1['ModeLifeCycleCount'] = [i.count[0] for i in f1['ModeLifeCycleName']]
f1['ModeLifeCycleName'] = [i.mode[0] for i in f1['ModeLifeCycleName']]

In [9]:

'''
Numpy doesn't natively perform arithmetic on dates like most SQL based applications. So
this little function converts the datetime objects in to 8 byte integers, gets their mean
and then converts that back into a datetime object and returns it
'''
def date_mean(date):
    return np.array(date, dtype='datetime64[s]').view('i8').mean().astype('datetime64[s]')
    
# get the average date of when the fee was charged
f5 = transactions_df.groupby('BBL').agg({'FeeIssuedDate': date_mean})
f1['MeanFeeIssueDate'] = f5

In [10]:
f1.head()

,TotalFeeAmount,NumberOfComplaints,ModeFeeTypeName,ModeFeeTypeCount,ModeLifeCycleName,ModeLifeCycleCount,MeanFeeIssueDate
BBL,,,,,,,
1000530006,16900,7,PROJECT BLDG,2,Building,7,2018-03-05 10:17:08
1000787510,200,1,Complaint,1,Building,1,2018-07-30 00:00:00
1000920003,200,1,VIOLATION,1,Building,1,2016-11-28 00:00:00
1001400003,5200,26,VIOLATION,26,Building,26,2019-01-30 02:46:09
1001640013,400,2,Complaint,2,Building,2,2018-11-01 12:00:00


In [16]:
#save file
f1.to_csv(r'./DHPFeesClean.csv', index='BBL', header=True)